In [ ]:
import requests
headstr="""Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Cookie: _ga=GA1.3.318223218.1528104516; _gid=GA1.3.1789114965.1531386334; JSESSIONID=9452E23A6BFE50BB9625955E8DF7E74E; __utma=72486415.318223218.1528104516.1531387568.1531387568.1; __utmc=72486415; __utmz=72486415.1531387568.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmb=72486415.1.10.1531387568
Host: plvr.land.moi.gov.tw
Referer: http://plvr.land.moi.gov.tw/DownloadOpenData
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"""



res=requests.get("http://plvr.land.moi.gov.tw//Download?type=zip&fileName=lvr_landcsv.zip")

In [ ]:
with open("realPriceLogin.zip",'wb') as f:
    f.write(res.content)

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("realPriceLogin.zip", 'r')
zip_ref.extractall("realPriceLogin")
zip_ref.close()

In [ ]:
import os 
import re
import pandas as pd

allData=[]
for i in os.listdir("realPriceLogin"):
    if len(i)==16 and i[-4:]==".CSV":
        with open("realPriceLogin/"+i,'r',encoding="utf8") as f:
            csvStrList=f.read().replace('\ufeff','').split("\n")
            keys=csvStrList[0].split(",")
            unitData=[]
            for j in csvStrList[2:]:
                data={}
                for k,v in zip(keys,j.split(",")):
                    data[k]=v
                unitData.append(data)
        allData.extend(unitData)
        print(len(unitData),i)

In [ ]:
allData[0]

In [ ]:
y=set()
usefulDatas=[]
for i in allData:
    if '土地區段位置/建物區段門牌' in i and i['單價(元/平方公尺)']!="" and (i['土地區段位置/建物區段門牌'][2]=="市" or i['土地區段位置/建物區段門牌'][2]=="縣"):
        if i['土地區段位置/建物區段門牌'] not in y and round(int(float(i['單價(元/平方公尺)'])*3.3058)/10000)!=0:
            usefulData={}
            usefulData['地址']=i['土地區段位置/建物區段門牌']
            usefulData['每坪價格']=str(round(int(float(i['單價(元/平方公尺)'])*3.3058)/10000))+"萬"
            usefulDatas.append(usefulData)
            y.add(i['土地區段位置/建物區段門牌'])

In [ ]:
import geocoder

In [ ]:
import geocoder
for i in usefulDatas:
    if i['地址'][2]=="市" or i['地址'][2]=="縣":
        n=0
        if 'lat' not in i:
            while n<5:
                try:
                    geoData=geocoder.arcgis(i['地址'])
                    i["lat"],i['lng']=geoData.latlng
                    n=10
                    print(i['地址'],i["lat"],i['lng'])
                    time.sleep(0.5)
                except:
                    n+=1
            

In [ ]:
for i in usefulDatas:
    i['bigadd'],i['smalladd']=re.findall("(\w\w[市|縣])(\w\w?\w?[區|市|鎮|鄉])",i['地址'])[0]

In [ ]:
import pymongo
# client=pymongo.MongoClient('192.168.1.113',27017,username='j122085',password='850605')
client=pymongo.MongoClient('localhost',27017,username='j122085',password='850605')
colle=client.rawData.realPriceLogin
# colle.drop()

In [ ]:
ids=[i['地址'] for i in usefulDatas]
from pymongo import UpdateOne
operations=[UpdateOne({"_id":idn},{"$set":data},upsert=True) for idn,data in zip(ids,usefulDatas)]

In [ ]:
colle.bulk_write(operations,ordered=False)

In [ ]:
colle.find_one({},{"_id":False})

In [11]:
from tkinter import Tk,StringVar,Label,Entry,Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait#直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import sys
import time 
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import time
def doAddPopChangeAssure():
    dataDir=e1.get()
    print('dataDir',dataDir)
    pathList=[(i[0]+"/"+j).replace("\\","/") for i in os.walk(dataDir) for j in i[2] if 'xls' in j]
    print('pathList',pathList)
    for excelFilePath in pathList:
        dirName=excelFilePath.split("/")[-2]
    #     dirName="30120315"
        while not (len(dirName) == 4 or len(dirName) == 7 or len(dirName) == 8):
            dirName = excelFilePath.split("/")[layer]
            layer -= 1
        print(dirName)
        try:
            dirName=dirName.split("/")[1]
        except:
            pass
        print(dirName)
        if len(dirName)==7:#民國年
            choice=dirName[:3]+"/"+dirName[3:5]+'/'+dirName[5:]
        elif len(dirName)==8:#西元年
            try:
                choice=str(int(dirName[:4])-1911)+"/"+dirName[4:6]+'/'+dirName[6:]
            except:
                pass
        elif len(dirName)==4:#只有月日
            try:
                choice=str(int(time.strftime("%Y"))-1911) +"/"+dirName[0:2]+'/'+dirName[2:]
            except:
                pass
        print(choice)
    
    
def selectPath():
    path_ = askdirectory()
    path.set(path_)
def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)
# with open('defaultIO.txt') as f:
#     IO=f.read()
# defaultLoadDir=IO.split("\n")[0].split("=")[1].replace("\\","/")
# defaultSaveDir=IO.split("\n")[1].split("=")[1].replace("\\","/")
root = Tk()
path = StringVar()
Label(root,text = "讀取資料夾:").grid(row = 0, column = 0)
e1=Entry(root, textvariable = path)
# e1.insert(0, defaultLoadDir)
e1.grid(row = 0, column = 1)
Button(root, text = "讀取資料夾", command = selectPath).grid(row = 0, column = 2)

path2 = StringVar()
Label(root,text = "存放資料夾:").grid(row = 1, column = 0)
e3=Entry(root, textvariable = path2)
# e3.insert(0, defaultSaveDir)
e3.grid(row = 1, column = 1)

Button(root, text = "存放資料夾", command = selectPath2).grid(row = 1, column = 2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column = 0)
e2 = Entry(root)
e2.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='加退調保開始', command=doAddPopChangeAssure).grid(row=3, column=1, pady=4)
root.mainloop()

dataDir Y:/1819/機器人上傳區/1070714
pathList ['Y:/1819/機器人上傳區/1070714/10210_退1.xls']
1070714
1070714
107/07/14


In [4]:
dataDir="Y:/1819/機器人上傳區"

In [9]:
[(i[0]+r"/"+j).replace("\\","/") for i in os.walk(dataDir) for j in i[2] if 'xls' in j]

['Y:/1819/機器人上傳區/1070714/10210_退1.xls']

In [8]:
for i in os.walk(dataDir):
    print(i)

('Y:/1819/機器人上傳區', ['1070714'], ['Thumbs.db'])
('Y:/1819/機器人上傳區\\1070714', [], ['10210_退1.xls'])
